# Beachwatch

This notebook examines the bacteria count data for the San Diego coastline, from the Beachwatch program. To analyze beachwatch data, we'll use the data package that is stored on the Library's data repository.

First, visit the [repository home page](http://data.sandiegodata.org) and note the tag for "water-project" below the search box. The [water-project](https://data.sandiegodata.org/dataset?tags=water-project) tag page lists all of the datasets for this project. In the (San Diego Beachwatch Data)[https://data.sandiegodata.org/dataset/ceden-waterboards-ca-gov-beachwatch-sandiego), Look for these to headings, just above the "Data and Resources" section:

- Loading the ZIP Package
- Loading the CSV Package

You can copy the code from one of those sections to get started. 

After opening the data package, we'll look at the results for the stations and station groups, a examine how well readons at one station are correlate with others in the same group. 


## Useful sites

* A great example of [mapping with geopandas](http://jonathansoma.com/lede/foundations-2017/classes/geopandas/mapping-with-geopandas/).


In [ ]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np

# Get the Package

Usually, the first thing you'll do with a Metatab data package is display the top level documentation, to see what resources it has and other basic information


In [ ]:
pkg = mp.open_package('http://library.metatab.org/ceden.waterboards.ca.gov-beachwatch-sandiego-2.zip')

pkg

In [ ]:
# Displaying a resource gives you the schema. This one isn't complete, since we havent filled in the column descriptions. 
pkg.resource('beachwatch-sd')

# Open the Resource

Below is another really common pattern. Get the resoruce and extract a Pandas DataFrame, using read_csv(). (You can also use ``.dataframe()``, which has more accurate datatypes, but is slower. ) We'll do some column modifications immediately, then display the data. 

In [ ]:
df = pkg.resource('beachwatch-sd').read_csv(parse_dates=True)

# It looks like the prefix of the station code groups stations, maybe into watersheds. 
df['stationgroup'] = df.stationcode.str[:2]

# The results has a large range, so log transformation makes them easier to visualize.
df['log_result'] = df.result.apply(np.log)

df.head()

In [ ]:
df[['stationcode','stationgroup']].drop_duplicates().groupby('stationgroup').count()

In [ ]:
df['stationcode'].value_counts().head()

In [ ]:
df['analyte'].value_counts().head()

To ensure that the following comparisions make sense, we'll want to focus on just one type of bacteria count

In [ ]:
df = df[df.analyte == 'Coliform, Total']

# Geographic Analysis

The Beachwatch data has position information for the stations, which we will need to match the stations to watersheds. Let's start be looking at their positions on a map. Shapely and Geopandas are the main tools for working with geographic data with Pandas and Jupyter. 


In [ ]:
from shapely.geometry import Point
import geopandas as gpd


## Create a new GeoPandas frame, converting the targetlongitude and targetlatitude
## colums to a Shapely Point and assigning it to the frame's geometry

gdf = gpd.GeoDataFrame(df, geometry=
                        [Point(x,y) for x,y in zip(df.targetlongitude, df.targetlatitude)])

# Here is a quick plot
gdf.plot()

In [ ]:

## Load a Metapack data package of the US Counties, then extract San Diego county by it's FIPS code, state=6
## county=73 
counties_pkg = mp.open_package('http://library.metatab.org/census.gov-counties-2017-2.csv')

# Use the Metapack feature for turning the Pandas dataframe into a GeoPandas dataframe
counties = counties_pkg.resource('counties').geoframe()

sd_county = counties[(counties.statefp==6) & (counties.countyfp==73) ]

In [ ]:
## Plot the county, then use the same Matplotlib axis to plot the points. 
base = sd_county.plot(color='white', edgecolor='black', figsize=(8*1.5,8))
gdf.plot(ax=base,  column='stationgroup', legend=True)
plt.title("Beachwatch Program Measurement Locations")
plt.show()

The 'EH' group seems really spread out, so let's have a closer look at just that group. 


In [ ]:
## Plot the county, then use the same Matplotlib axis to plot the points. 
base = sd_county.plot(color='white', edgecolor='black', figsize=(8*1.5,8))
gdf[(gdf.stationgroup == 'EH') ].plot(ax=base,  column='stationgroup', legend=True)
plt.title("Beachwatch Program Measurement Locations for EH group")
plt.show()

The 'EH' group is al over the coast, so the stations in that group probably wont correlate with each otehr very well. Let's exclude it. 


In [ ]:
df = df[df.stationgroup != 'EH']

In [ ]:
fig, ax = plt.subplots(1,figsize=(18,6))
df[(df.stationgroup == 'SE') & (df.sampledate.dt.year == 2004)].groupby('stationcode').plot(ax=ax, x='sampledate', y='result')
ax.set_yscale("log", nonposy='clip')

In [ ]:
fig, ax = plt.subplots(1,figsize=(18,6))
df[(df.stationcode=='IB-080') & (df.sampledate.dt.year > 2003) & (df.sampledate.dt.year < 2008)].set_index('sampledate').resample('1m').mean().plot(ax=ax, y='result')
ax.set_yscale("log", nonposy='clip')

In [ ]:
fig, ax = plt.subplots(1,figsize=(18,6))
df[(df.stationgroup=='IB') & (df.sampledate.dt.year > 2003) & (df.sampledate.dt.year < 2008)].set_index('sampledate').groupby(['stationname',pd.Grouper(freq='m')])\
    .mean().plot(ax=ax, y='result')
ax.set_yscale("log", nonposy='clip')

In [ ]:
df[(df.stationgroup=='IB') & (df.result < 100)].result.hist(bins=100, log=True)

# Station group correlations


It is likely that since stations within a group are close to each other, the measures for one group are similar to others in the same group. So, we should try to characterize how well readings between stations in a group are correlated. 


In [ ]:
groups = list(df.stationgroup.unique())
fig, axes = plt.subplots(len(groups), 1,figsize=(18,5*len(groups)))

for ax, group in zip(axes, groups):
    
    _ = df[(df.stationgroup==group)]\
        .set_index('sampledate').groupby(['stationcode',pd.Grouper(freq='m')]).mean()
    _.reset_index().set_index('sampledate').groupby('stationcode').plot(ax=ax,y='result', legend = False)
    ax.set_yscale("log", nonposy='clip')
    ax.set_title("Beachwatch station group {}".format(group))

    
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    


In [ ]:
from IPython.display import display

def mean_correlation(df, group, column='result'):
    """Build the corelation matrix for all columns, remove the diagonal, and average the remaining values. This uses
    the full matrix, not the triangular matrix, so each value appears twice. """
    
    _ = df[(df.stationgroup==group)]\
            .set_index('sampledate').groupby(['stationcode',pd.Grouper(freq='m')]).mean()
    _ = _.reset_index().set_index(['stationcode','sampledate'])[column].unstack(level=0)
    corr = _.corr().stack().to_frame()
    corr.columns = [column]
    
    _ = corr[corr[column] < 1.0]

    
    return _.mean().iloc[0]


In [ ]:
pd.DataFrame([ (group, mean_correlation(df, group,'result') ) for group in groups ]).sort_values(1, ascending=False)
    

In [ ]:
pd.DataFrame([ (group, mean_correlation(df, group,'log_result') ) for group in groups ]).sort_values(1, ascending=False)
   